In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import time
import math
from datetime import datetime
root=''
try:
  from google.colab import drive
  drive.mount('/content/drive')
  root='./drive/MyDrive/gnn/data/'
except:
  pass
device='cuda' if torch.cuda.is_available() else 'cpu'
print(torch.cuda.get_device_name(0))
torch.cuda.empty_cache()

In [ ]:
filename=root+'data_118quad/118dc_quad_ISF.txt'
S_isf=pd.read_table(filename,sep=',',header=None).to_numpy() # ISF matrix
filename=root+'data_118quad/118ac_fmax.txt'
f_max=pd.read_table(filename,sep=',',header=None).to_numpy() # flow limit
n_line = np.size(S_isf,0)
S = torch.from_numpy(S_isf).to(device) # ISF
f_max = torch.from_numpy(f_max).to(device) # flow limit
print(S.shape,f_max.shape,torch.min(f_max))

x=np.load(root+'ac118_p10_x_v.npy')
y=np.load(root+'ac118_p10_y_v.npy')
W=np.load(root+'data_118_quad/ac118_p10_w.npy')
print(np.max(x),np.min(x),np.max(y),np.min(y))

# scaling on voltage
vy_deviation = 0.9
vy_scale = 100
y[:,1,:] = (y[:,1,:] - vy_deviation) * vy_scale
print('voltage range(scaled):',np.min(y[:,1,:]),np.max(y[:,1,:]))

# scaling on price
pi_deviation = 0
y[:,0,:] = y[:,0,:] + pi_deviation
# filter out extreme points in price
y_sort_arg = np.argsort(np.amax(np.abs(y[:,0,:]),axis=0)) # max extreme
y_sort_arg1 = np.argsort(np.amin(y[:,0,:],axis=0),axis=0) # min extreme

del_idx0 = []
del_num = 0
for i in range(del_num):
  del_idx0.append(y_sort_arg[-i])
  del_idx0.append(y_sort_arg1[i])
# print(del_idx0)
del_idx = [] # keep only non-repeated
[del_idx.append(x) for x in del_idx0 if x not in del_idx]
del_idx = np.sort(del_idx)
# delete extreme points
print('price range old:',np.min(y[:,0,:]),np.max(y[:,0,:]))
print('voltage range old:',np.min(y[:,1,:]),np.max(y[:,1,:]))
x = np.delete(x, del_idx, axis=2)
y = np.delete(y, del_idx, axis=2)
print('price range new:',np.min(y[:,0,:]),np.max(y[:,0,:]))
print('voltage range new:',np.min(y[:,1,:]),np.max(y[:,1,:]))
print(x.shape,y.shape)

In [ ]:
n_sample=y.shape[-1]
n_bus=y.shape[0]
x_total=x.transpose((1,0,2)).reshape(-1,x.shape[-1])
y_total=y.transpose((1,0,2)).reshape(-1,y.shape[-1])
x_train,x_test,y_train,y_test=train_test_split(x_total.T,y_total.T,test_size=0.2)
class Dataset(torch.utils.data.Dataset):
    def __init__(self,x,y):
        self.x=torch.from_numpy(x).float()
        self.y=torch.from_numpy(y).float()
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
          idx=idx.tolist()
        return self.x[idx],self.y[idx]
params={'batch_size': 256,
        'shuffle': True,
        'num_workers': 0}
train=Dataset(x_train,y_train)
train_set=torch.utils.data.DataLoader(train,**params)
val=Dataset(x_test,y_test)
val_set=torch.utils.data.DataLoader(val,**params)

In [ ]:
# fig2 = plt.figure(figsize=(8,4))
# flat_list = list(np.concatenate(y[:,n_sample:]).flat)
# flat_list3 = list(np.concatenate(y[:,:n_sample]).flat)
# plt.subplot(1,2,1)
# plt.hist(flat_list,bins = 100,label = 'voltage')
# plt.subplot(1,2,2)
# # plt.hist(flat_list3,range=[-2000, 2000],bins = 100,label = 'price')
# plt.hist(flat_list3,bins = 100,label = 'price')
# plt.show()

In [ ]:
class dnn(torch.nn.Module):
  def __init__(self,shape):
    super(dnn,self).__init__()
    layers=[]
    for idx in range(len(shape)-2):
      layers.extend([
        nn.Linear(shape[idx],shape[idx+1]),
        nn.BatchNorm1d(shape[idx+1]),
        nn.ReLU(),
        nn.Dropout(0.5),
      ])
    layers+=[nn.Linear(shape[-2],shape[-1])]
    self.features=nn.Sequential(*layers)
    for temp in self.features:
      if type(temp)==nn.Linear:
        torch.nn.init.normal_(temp.weight,mean=0,std=1)
  def forward(self,x): return self.features(x)
net=dnn([n_bus*6,n_bus*10,n_bus*10,n_bus*6,n_bus*2]).to(device)
print('number of params: %d'%(sum(temp.numel() for temp in net.parameters() if temp.requires_grad)))

In [ ]:
# threshold function for p_g
class my_gen_pred_binary(nn.Module):
  def __init__(self):
    super(my_gen_pred_binary,self).__init__()
  def forward(self,x,thresh):
    right_thresh=thresh.clone().detach().requires_grad_(True).double()
    left_thresh=torch.tensor(0).double()
    x=x.double()
    output = torch.sigmoid(left_thresh - x)
    output = torch.mul(output,left_thresh - x) + x
    output = torch.sigmoid(output - right_thresh)
    output = torch.mul(output,output - right_thresh) + right_thresh
    return output

In [ ]:
## params needed for S calculation
# line parameters
filename1 = '/content/drive/MyDrive/gnn/data/data_118_quad/ieee118_lineloc.txt'
filename2 = '/content/drive/MyDrive/gnn/data/data_118_quad/ieee118_lineparams.txt'
filename3 = '/content/drive/MyDrive/gnn/data/data_118_quad/ieee118_Bmat.txt'
# incidence info
line_loc = pd.read_table(filename1,sep=',',header=None).to_numpy()
# r, x, shunt, S_max
line_params = pd.read_table(filename2,sep=',',header=None).to_numpy()
B_mat=pd.read_table(filename3,sep=',',header=None).to_numpy()
B_r = np.delete(B_mat,68,axis=0)
B_r = np.delete(B_r,68,axis=1)
Br_inv = np.linalg.inv(B_r)

R_line = line_params[:,0].copy()
X_line = line_params[:,1].copy()

B_shunt = line_params[:,2].copy()

Z_line = R_line + 1j * X_line 
Y_line = 1 / Z_line

G_line = np.real(Y_line)
B_line = np.imag(Y_line)

# transformer indicator
a = (R_line > 0).astype(int)

# params to tensor and GPU
G_line_tensor = torch.from_numpy(G_line).to(device) # conductance
B_line_tensor = torch.from_numpy(B_line).to(device) # susceptance
B_shunt_tensor = torch.from_numpy(B_shunt/2).to(device) # conductance
Br_inv_tensor = torch.from_numpy(Br_inv).to(device) # reduced Bbus matrix
a_tensor = torch.from_numpy(a).double().to(device) # line/transformer

In [ ]:
class loss_func:
    def __init__(self,s_max,G_line,B_line,B_shunt,Br_inv,a,line_loc,Jpf,theta_n,pinj0,vy_scale,vy_deviation,pi_deviation):
      self.s=s_max
      self.g=G_line
      self.b=B_line
      self.c=B_shunt
      self.r=Br_inv
      self.a=a
      self.J1=Jpf
      self.t0=theta_n
      self.p0=pinj0
      self.vy_s=vy_scale
      self.vy_d=vy_deviation
      self.vy_pd=pi_deviation
      self.mse=nn.MSELoss() # MSE loss
      self.lmda1=torch.tensor(10).to(device) # V MSE 
      self.lmda2=torch.tensor(1).to(device) # pi MSE 
      self.lmda3=torch.tensor(0.1).to(device) # v l_inf
      self.lmda4=torch.tensor(0.1).to(device) # s feasibility
      self.lmda5=torch.tensor(0.01).to(device) # pi l_inf
      self.line_loc=line_loc
      self.binary_cell=my_gen_pred_binary()
    def calc(self,pred,label,x,feas):
      mse_p=self.mse(pred[:,:n_bus],label[:,:n_bus])
      mse_v=self.mse(pred[:,n_bus:],label[:,n_bus:])
      linf_p=(pred[:,:n_bus]-label[:,:n_bus]).norm(p=float('inf'))
      linf_v=(pred[:,n_bus:]-label[:,n_bus:]).norm(p=float('inf'))
      if feas==False:
        return self.lmda1*mse_v+self.lmda2*mse_p+self.lmda3*linf_v+self.lmda5*linf_p
      label_pred=pred[:,:n_bus]-self.vy_pd #128,1354
      p_max=x[:,:n_bus*1]-x[:,n_bus*1:n_bus*2]
      quadratic_b=x[:,n_bus*4:n_bus*5]
      quadratic_a=x[:,n_bus*5:n_bus*6]
      quadratic_center=(label_pred-quadratic_b)/(quadratic_a+1e-5)/2
      p_inj=self.binary_cell(quadratic_center,p_max)
      bus_inj=p_inj+x[:,n_bus:n_bus*2]
      p_inj_r=torch.cat((bus_inj[:,:639],bus_inj[:,640:]),1)/100
      p_inj_dev=p_inj_r-self.p0
      theta0=torch.matmul(self.J1,p_inj_dev.T)
      ref_ang=torch.zeros(1,theta0.shape[1]).to(device)
      theta=torch.cat([theta0[:639,:],ref_ang,theta0[639:,:]],0)+self.t0 # deviation +nominal, slack 639
      v_pred=pred[:,n_bus:].transpose(0,1)/self.vy_s+self.vy_d
      
      # s penalty
      theta1=theta[self.line_loc[:,0]-1,:]
      theta2=theta[self.line_loc[:,1]-1,:]
      V1=(v_pred[self.line_loc[:,0]-1,:]).double()
      V2=(v_pred[self.line_loc[:,1]-1,:]).double()
      f_p=(self.a*self.g*(V1*V1).T)-self.a*((V1*V2).T)*(self.g*torch.cos(theta1-theta2).T+self.b*torch.sin(theta1-theta2).T)
      f_p=f_p.T
      f_q=-self.a*(V1.T)*(self.a*V1.T)*(self.b+self.c/2)+self.a*((V1*V2).T)*(self.b*torch.cos(theta1-theta2).T-self.g*torch.sin(theta1-theta2).T)
      f_q=f_q.T
      s_pred=torch.sqrt(f_p*f_p+f_q*f_q)*100
      s_penalty=torch.sigmoid(s_pred-self.s)+torch.sigmoid(-s_pred-self.s)
      s_total=torch.sum(s_penalty)

      # sji penalty
      theta1=theta[self.line_loc[:,1]-1,:]
      theta2=theta[self.line_loc[:,0]-1,:]
      V1=(v_pred[self.line_loc[:,1]-1,:]).double()
      V2=(v_pred[self.line_loc[:,0]-1,:]).double() 
      fji_p=(self.a*self.g*(V1*V1).T)-self.a*((V1*V2).T)*(self.g*torch.cos(theta1-theta2).T+self.b*torch.sin(theta1-theta2).T)
      fji_p=fji_p.T
      fji_q=-self.a*(V1.T)*(self.a*V1.T)*(self.b+self.c/2)+self.a*((V1*V2).T)*(self.b*torch.cos(theta1-theta2).T-self.g*torch.sin(theta1-theta2).T)
      fji_q=fji_q.T
      sji_pred=torch.sqrt(fji_p*fji_p+fji_q*fji_q)*100
      sji_penalty=torch.sigmoid(sji_pred-self.s)+torch.sigmoid(-sji_pred-self.s)
      sji_total=torch.sum(sji_penalty)

      return self.lmda1*mse_v+self.lmda2*mse_p+self.lmda3*linf_v+self.lmda5*linf_p+self.lmda4*s_total+self.lmda4*sji_total
my_loss=loss_func(f_max,G_line_tensor,B_line_tensor,B_shunt_tensor,Br_inv_tensor,a_tensor,line_loc,Jpf_tensor,theta0_tensor,pinj0_tensor,vy_scale_tensor,vy_deviation_tensor,pi_deviation_tensor)

In [ ]:
path=root+'data_1354ac_2022/gnn_trained_ac1354.pickle'
try: 
  net.load_state_dict(torch.load(path))
  print('params loaded')
except: 
  print('cold start')

optimizer=torch.optim.Adam(net.parameters())
train_loss=[]
val_loss=[]

## Training
t0=time.time()
max_epochs=100
eval_epoch=5

# earlystopping
tolerance=5
min_delta=1e-3
previous=0

# add feasibility 
feas=True
for epoch in range(max_epochs):
  # training loop
  total_loss=0.0
  for local_batch,local_label in train_set:
    optimizer.zero_grad() # clear the past gradient
    local_batch,local_label=local_batch.to(device),local_label.to(device)
    logits=net(local_batch)
    loss=my_loss.calc(logits,local_label,local_batch,feas)
    loss.backward()
    optimizer.step()
    total_loss+=loss.item()
  avg_loss=total_loss/len(train_set.dataset)
  train_loss.append(avg_loss)
  print("Epoch %d | Training loss: %.4f"%(epoch,avg_loss))
  # eval
  if epoch%eval_epoch==0:
    net.eval()
    total_loss=0.0
    for local_batch,local_label in val_set:
      local_batch,local_label=local_batch.to(device),local_label.to(device)
      logits=net(local_batch)
      loss=my_loss.calc(logits,local_label,local_batch,feas)
      total_loss+=loss.item()
    avg_loss=total_loss/len(val_set.dataset)
    val_loss.append([epoch, avg_loss])
    print("Epoch %d | Testing loss: %.4f"%(epoch,avg_loss))
    if epoch:
      if previous-avg_loss<min_delta: tolerance-=1
      if tolerance==0: pass
    previous=avg_loss
    net.train()
t1=time.time()
print("Training time:%.4fs"%(t1-t0))

In [ ]:
timestamp=datetime.now().strftime('%m%d%H%M')
path=root+'data_1354ac_2022/feasdnn0411_%s.pickle'%(timestamp)
if feas==False: path.replace('feas','')
print(path)
torch.save(net.state_dict(),path)

In [ ]:
val_len = len(val_loss)
print(val_len)
val_plt = np.zeros((2,val_len))
for i in range(val_len):
  val_plt[0,i] = val_loss[i][0]
  val_plt[1,i] = val_loss[i][1]

# plt.figure()
# plot_idx = np.arange(np.size(train_loss))
# plt.plot(plot_idx[5:-1],train_loss[5:-1],lw=2,label='training loss')
# plt.plot(val_plt[0,1:],val_plt[1,1:],lw=2,label='validation loss')
# plt.yscale("log")
# plt.xlabel('epoch')
# plt.ylabel('loss')
# plt.legend()
# plt.show(block=False)

# Evaluate the model w/ validation set

In [ ]:
n_test = np.size(x_test,0)
x_test_feed = torch.from_numpy(x_test).float()
x_test_feed = x_test_feed#.transpose(1,2)
x_test_feed = x_test_feed.to(device)
print('Validation dataset size:',x_test_feed.shape)
print('Number of validation set: ',n_test)
y_pred = net(x_test_feed)
print(y_pred.shape)

* Visualization




 - Visualize errors

In [ ]:
y_pred1 = y_pred.cpu().detach()
y_pred1 = torch.squeeze(y_pred1,1).numpy()#.transpose()
print(y_test.shape,y_pred1.shape)

In [ ]:
# x=np.reshape(x,(x.shape[0]*x.shape[1],x.shape[2])) # reshape by samples not dim1
# y=np.reshape(y,(y.shape[0]*y.shape[1],y.shape[2]))
# print(x_pre.shape,y_pre.shape)

y_pred_temp = y_pred1.copy().transpose()
# y_pred2=np.reshape(y_pred2,(y_pre.shape[0],y_pre.shape[1],n_test))
y_pred2=np.zeros([y.shape[0],y.shape[1],n_test])
y_pred2[:,0,:]=y_pred_temp[:n_bus,:]
y_pred2[:,1,:]=y_pred_temp[n_bus:,:]
print(y_pred2.shape,y_pred1.shape)

y_test_temp = y_test.copy().transpose()
# y_pred2=np.reshape(y_pred2,(y_pre.shape[0],y_pre.shape[1],n_test))
y_test2=np.zeros([y.shape[0],y.shape[1],n_test])
y_test2[:,0,:]=y_test_temp[:n_bus,:]
y_test2[:,1,:]=y_test_temp[n_bus:,:]
y_pred1 = y_pred2.copy()

In [ ]:
# recover the original p.u. scale
# vy_deviation) * vy_scale
y_pred1[:,1,:] = y_pred1[:,1,:] / vy_scale + vy_deviation
y_test2[:,1,:] = y_test2[:,1,:] / vy_scale + vy_deviation
print(y_test2.shape,y_pred1.shape)

In [ ]:
n_test = np.size(y_test2,2)
err_L2 = np.zeros(n_test)
err_Linf = np.zeros(n_test)
for i in range(n_test):
  err_L2[i] = np.linalg.norm(y_test2[:,0,i] - y_pred1[:,0,i]) / np.linalg.norm(y_test2[:,0,i])
  err_Linf[i] = np.max(np.abs(y_test2[:,0,i] - y_pred1[:,0,i])) / np.max(np.abs(y_test2[:,0,i]))

err_L2_v = np.zeros(n_test)
err_Linf_v = np.zeros(n_test)
for i in range(n_test):
  err_L2_v[i] = np.linalg.norm(y_test2[:,1,i] - y_pred1[:,1,i]) / np.linalg.norm(y_test2[:,1,i])
  err_Linf_v[i] = np.max(np.abs(y_test2[:,1,i] - y_pred1[:,1,i])) / np.max(np.abs(y_test2[:,1,i]))

In [ ]:
err_L2_mean = np.mean(err_L2)
err_Linf_mean = np.mean(err_Linf)
print('Price L2 mean:', err_L2_mean,'L_inf mean:', err_Linf_mean )
err_L2_mean_v = np.mean(err_L2_v)
err_Linf_mean_v = np.mean(err_Linf_v)
print('Voltage L2 mean:', err_L2_mean_v,'L_inf mean:', err_Linf_mean_v )

# fig2 = plt.figure(figsize=(16, 16))
# plt.subplot(2, 2, 1)
# # plt.hist(np.abs(ga),bins = 10)
# plt.plot(err_L2,'bo',markersize=0.5,label = 'L2 error')
# plt.plot(err_Linf,'r^',markersize=0.5,label = 'Linf error')
# plt.legend(loc="upper right")
# plt.xlabel('sample index')
# plt.ylabel('error')
# plt.title('Normalized sample error')
# plt.grid(True)
# # error histogram
# plt.subplot(2, 2, 2)
# plt.hist(err_L2, bins = 50, facecolor='b', alpha=0.75,label = 'L2 error')
# plt.hist(err_Linf, bins = 50, facecolor='g', alpha=0.75,label = 'Linf error')
# plt.legend(loc="upper right")
# plt.xlabel('sample error value')
# plt.ylabel('frequency')
# plt.title('Histogram of L_2 error')
# # plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
# plt.grid(True)

# plt.subplot(2, 2, 3)
# # plt.hist(np.abs(ga),bins = 10)
# plt.plot(err_L2_v,'bo',markersize=0.5,label = 'L2 error')
# plt.plot(err_Linf_v,'r^',markersize=0.5,label = 'Linf error')
# plt.legend(loc="upper right")
# plt.xlabel('sample index')
# plt.ylabel('error')
# plt.title('Normalized sample error')
# plt.grid(True)
# # error histogram
# plt.subplot(2, 2, 4)
# plt.hist(err_L2_v, bins = 50, facecolor='b', alpha=0.75,label = 'L2 error')
# plt.hist(err_Linf_v, bins = 50, facecolor='g', alpha=0.75,label = 'Linf error')
# plt.legend(loc="upper right")
# plt.xlabel('sample error value')
# plt.ylabel('frequency')
# plt.title('Histogram of L_2 error')
# # plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
# plt.grid(True)
# plt.show()

In [ ]:
print(y_pred1[:,1,:n_test].shape,y_test2[:,1,:n_test].shape)
print('true range:',np.max(y_test2[:,1,:n_test]),np.min(y_test2[:,1,:n_test]))
print('predicted range',np.max(y_pred1[:,1,:n_test]),np.min(y_pred1[:,1,:n_test]))

# fig3 = plt.figure(figsize=(16, 8))
# flat_list1 = list(np.concatenate(y_test2[:,1,:n_test]).flat)
# flat_list2 = list(np.concatenate(y_pred1[:,1,:n_test]).flat)
# plt.hist(flat_list1,bins = 100,label = 'true')

# plt.hist(flat_list2,bins = 100,label = 'pred')
# plt.legend(loc="upper right")
# plt.grid(True)
# plt.show()

In [ ]:
print(x.shape,n_sample)

x_new = np.zeros([x.shape[0],x.shape[1],n_sample])
for i in range(x.shape[1]):
  x_new[:,i,:] = x_total[n_bus*i:n_bus*(i+1),:]

y_new = np.zeros([y.shape[0],y.shape[1],n_sample])
for i in range(y.shape[1]):
  y_new[:,i,:] = y_total[n_bus*i:n_bus*(i+1),:]

# Predict generation using $\pi$
* Using predicted $\pi$ and find the active constraints in $p_G(i)$
* For inactive $p_G(i)$ consider other methods like power flow balance

In [ ]:
gen_limit0 = x_new[:,4,:].copy() # lin cost
print(gen_limit0.shape)

gen_idx = []
gen_idx = np.arange(n_bus)
# for i in range(n_bus):
#   if gen_limit0[i,0] > 0:
#     gen_idx.append(i)
print(type(gen_idx),len(gen_idx),gen_idx)

In [ ]:
n_sample=x_total.shape[-1]
x_feed = torch.from_numpy(x_total.T).float()
y_pred1=net(x_feed.to(device)).cpu().detach().numpy().T
y_pred_temp = y_pred1.copy()
y_pred2=np.zeros([y.shape[0],y.shape[1],n_sample])
y_pred2[:,0,:]=y_pred_temp[:n_bus,:]
y_pred2[:,1,:]=y_pred_temp[n_bus:,:]
print(y_pred2.shape,y_pred1.shape)
y_pred1 = y_pred2.copy()

In [ ]:
gen_cost0 = x_new[:,4,:].copy()
lmp_data = y_new[:,0,:].copy()
quadratic_a = x_new[:,5,:].copy()
profit_pred = y_pred1[:,0,:] - gen_cost0
print(np.min(np.abs(profit_pred)))
profit_true = lmp_data - gen_cost0
print(np.min(np.abs(profit_true)))
profit_pred=(y_pred1[:,0,:]-gen_cost0)/(quadratic_a+1e-10)/2
profit_true=(lmp_data-gen_cost0)/(quadratic_a+1e-10)/2
print(np.min(np.abs(profit_pred)))
print(np.min(np.abs(profit_true)))

In [ ]:
print(profit_pred.shape,profit_true.shape)
profit_err = profit_true - profit_pred
profit_err_l2 = np.zeros([n_sample,1])

for i in range(n_sample):
  profit_err_l2[i] = np.linalg.norm(profit_err[:,i])/np.linalg.norm(profit_true[:,i])
print(np.mean(profit_err_l2))

# fig5 = plt.figure(figsize=(16, 8))
# # error histogram
# plt.hist(profit_err_l2, bins = 50, facecolor='b', alpha=0.75,label = 'L2 error')
# # plt.hist(err_linf_new, bins = 50, facecolor='g', alpha=0.75,label = 'Linf error')
# plt.legend(loc="upper right")
# plt.xlabel('sample error value')
# plt.ylabel('frequency')
# plt.title('Histogram of L_2 error')
# # plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
# plt.grid(True)
# plt.show()

In [ ]:
p_pred_sort = np.reshape(profit_pred,n_bus*n_sample)
p_true_sort = np.reshape(profit_true,n_bus*n_sample)
print(p_pred_sort.shape)
print(np.min(p_pred_sort),np.min(p_true_sort))

In [ ]:
# fig2 = plt.figure(figsize=(8, 8))
# plt.hist(p_pred_sort, bins = 50, facecolor='b', alpha=0.75,label = 'pred. profit')
# plt.hist(p_true_sort, bins = 50, facecolor='g', alpha=0.75,label = 'true profit')
# plt.legend(loc="upper right")
# plt.xlabel('value')
# plt.ylabel('frequency')
# plt.title('profit histogram')
# # plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
# plt.grid(True)
# plt.show()

In [ ]:
# x = [load, gen_cost, gen_lim]
binary_thres_true = 1e-5
binary_thres = x_new[:,0,:].copy() # upper
binary_thres_lo = x_new[:,1,:].copy() # lower
gen_pred_binary_full = np.zeros((n_bus,n_sample))
gen_true_binary_full = np.zeros((n_bus,n_sample))

for i in range(n_sample):
  for j in range(len(gen_idx)):
    # predicted generator limit
    if profit_pred[gen_idx[j],i] > binary_thres[gen_idx[j],i]:
      gen_pred_binary_full[gen_idx[j],i] = binary_thres[gen_idx[j],i]
    elif profit_pred[gen_idx[j],i] < binary_thres_lo[gen_idx[j],i]:
      gen_pred_binary_full[gen_idx[j],i] = binary_thres_lo[gen_idx[j],i]
    else:
      gen_pred_binary_full[gen_idx[j],i] = profit_pred[gen_idx[j],i]
    # true generator limit
    if profit_true[gen_idx[j],i] > binary_thres[gen_idx[j],i]:
      gen_true_binary_full[gen_idx[j],i] = binary_thres[gen_idx[j],i]
    elif profit_true[gen_idx[j],i] < binary_thres_lo[gen_idx[j],i]:
      gen_true_binary_full[gen_idx[j],i] = binary_thres_lo[gen_idx[j],i]
    else:
      gen_true_binary_full[gen_idx[j],i] = profit_true[gen_idx[j],i]

gen_inj=gen_pred_binary_full
gen_inj_true=gen_true_binary_full
# nodal injection
load0 = -x_new[:,1,:].copy() # load file
p_inj = gen_inj #- load0
p_inj_true = gen_inj_true #- load0
print(np.sum(p_inj),np.sum(gen_inj_true))
print(np.sum(p_inj),np.sum(load0),np.sum(gen_inj))

## injection accuracy

In [ ]:
print(p_inj_true.shape,p_inj.shape)
p_inj_true_sort = np.reshape(p_inj_true,n_bus*n_sample)
p_inj_sort = np.reshape(p_inj,n_bus*n_sample)
p_err = np.zeros(n_sample)
for i in range(n_sample):
  p_err[i] = np.linalg.norm(p_inj_true[:,i]-p_inj[:,i]) / np.linalg.norm(p_inj_true[:,i])
print('mean p_inj l2 err:',np.mean(p_err))
# fig3 = plt.figure(figsize=(16, 8))
# plt.subplot(1,2,1)
# plt.hist(p_inj_sort, bins = 50, facecolor='b', alpha=0.75,label = 'pred. injection')
# plt.hist(p_inj_true_sort, bins = 50, facecolor='g', alpha=0.75,label = 'true injection')
# plt.legend(loc="upper right")
# plt.xlabel('value')
# plt.ylabel('frequency')
# plt.title('injection histogram')
# plt.subplot(1,2,2)
# plt.hist(p_err, bins = 50, facecolor='b', alpha=0.75,label = 'injection err')
# plt.legend(loc="upper right")
# plt.xlabel('value')
# plt.ylabel('frequency')
# plt.title('error histogram')
# # plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
# plt.grid(True)
# plt.show()

* Calculate flow

In [ ]:
filename=root+'data_1354ac_2022/ieee1354_fmax.txt'
f_max1=pd.read_table(filename,sep=',',header=None).to_numpy() # flow limit

n_line = np.size(S_isf,0)
flow_est = np.zeros((n_line,n_sample))
flow_est0 = np.zeros((n_line,n_sample))

f_binary = np.zeros((n_line,n_sample))
f_binary0 = np.zeros((n_line,n_sample))

# for i in range(n_sample):
flow_est = np.dot(S_isf,p_inj)
flow_est0 = np.dot(S_isf,p_inj_true)
# f_max
# f_max_numpy = f_max.cpu().detach().numpy()
f_max_numpy = f_max1.copy()
f_binary = (np.abs(flow_est)-f_max_numpy > 0)
f_binary0 = (np.abs(flow_est0)-f_max_numpy > 0)

print(f_max_numpy.shape,flow_est.shape,flow_est0.shape)
f_tot_sample = n_line * n_sample
print(np.sum(f_binary),np.sum(f_binary0))
print(np.sum(f_binary)/f_tot_sample,np.sum(f_binary0)/f_tot_sample)
print(n_line,n_sample,flow_est.shape)

In [ ]:
# soft threshold
f_err_est = np.abs(flow_est)-f_max_numpy
f_err_true = np.abs(flow_est0)-f_max_numpy

f_err_est = np.maximum(np.abs(flow_est)-f_max_numpy,0) # identify violations
f_err_true = np.maximum(np.abs(flow_est0)-f_max_numpy,0)

print(np.max(f_err_est),np.max(f_err_true))
print(np.max(f_err_est/f_max_numpy),np.max(f_err_true/f_max_numpy))

In [ ]:
f_binary_soft = (np.abs(flow_est)-f_max_numpy > 0.1*(f_max_numpy))
f_binary0_soft = (np.abs(flow_est0)-f_max_numpy > 0.1*(f_max_numpy))
print(np.sum(f_binary_soft),np.sum(f_binary0_soft))
print(np.sum(f_binary_soft)/f_tot_sample,np.sum(f_binary0_soft)/f_tot_sample)

In [ ]:
f_pred_sort = np.reshape(f_err_est/f_max_numpy,n_line*n_sample)
f_true_sort = np.reshape(f_err_true/f_max_numpy,n_line*n_sample)

# fig2 = plt.figure(figsize=(16, 8))
# plt.subplot(1,2,1)
# plt.hist(f_pred_sort, bins = 10, facecolor='b', alpha=0.75,label = 'pred. f')
# plt.hist(f_true_sort, bins = 10, facecolor='g', alpha=0.75,label = 'true f')
# plt.legend(loc="upper right")
# plt.xlabel('percentage')
# plt.ylabel('frequency')
# plt.title('flow violation level histogram')
# plt.subplot(1,2,2)
# plt.hist(f_true_sort, bins = 10, facecolor='g', alpha=0.75,label = 'true f')
# plt.legend(loc="upper right")
# plt.xlabel('percentage')
# plt.ylabel('frequency')
# plt.title('flow violation level histogram')
# # plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
# plt.grid(True)
# plt.show()

In [ ]:
f_line = np.sum(f_binary,0)
f_samp = np.sum(f_binary,1)
print('max sample pred:',np.max(f_line))
print('max line pred:',np.max(f_samp))

f_line0  = np.sum(f_binary0,0)
f_samp0 = np.sum(f_binary0,1)
print('max sample true:',np.max(f_line0))
print('max line true:',np.max(f_samp0))

# Check objective optimality

In [ ]:
gen_cost_pred = np.zeros((n_bus,n_sample))
gen_cost_true = np.zeros((n_bus,n_sample))
objective_err = np.zeros(n_sample)

gen_cost_pred = np.multiply(np.multiply(p_inj,p_inj),quadratic_a) + np.multiply(p_inj,gen_cost0)
gen_cost_true = np.multiply(np.multiply(p_inj_true,p_inj_true),quadratic_a) + np.multiply(p_inj_true,gen_cost0)

objective_err = np.sum(np.abs(gen_cost_true-gen_cost_pred),axis=0) / np.sum(gen_cost_true,axis=0)
print(np.mean(objective_err))

# fig6 = plt.figure(figsize=(16, 8))
# # error histogram
# plt.hist(objective_err, bins = 50, facecolor='b', alpha=0.75,label = 'L2 error')
# # plt.hist(err_linf_new, bins = 50, facecolor='g', alpha=0.75,label = 'Linf error')
# plt.legend(loc="upper right")
# plt.xlabel('sample error value')
# plt.ylabel('frequency')
# plt.title('Histogram of L_2 error')
# # plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
# plt.grid(True)
# plt.show()

## injection accuracy

In [ ]:
print(p_inj_true.shape,p_inj.shape)

p_inj_true_sort = np.reshape(p_inj_true,n_bus*n_sample)
p_inj_sort = np.reshape(p_inj,n_bus*n_sample)

p_err = np.zeros(n_sample)
for i in range(n_sample):
  p_err[i] = np.linalg.norm(p_inj_true[:,i]-p_inj[:,i]) / np.linalg.norm(p_inj_true[:,i])

print('mean p_inj l2 err:',np.mean(p_err))
# fig3 = plt.figure(figsize=(16, 8))
# plt.subplot(1,2,1)
# plt.hist(p_inj_sort, bins = 50, facecolor='b', alpha=0.75,label = 'pred. injection')
# plt.hist(p_inj_true_sort, bins = 50, facecolor='g', alpha=0.75,label = 'true injection')
# plt.legend(loc="upper right")
# plt.xlabel('value')
# plt.ylabel('frequency')
# plt.title('injection histogram')
# plt.subplot(1,2,2)
# plt.hist(p_err, bins = 50, facecolor='b', alpha=0.75,label = 'injection err')
# plt.legend(loc="upper right")
# plt.xlabel('value')
# plt.ylabel('frequency')
# plt.title('error histogram')
# # plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
# plt.grid(True)
# plt.show()

# Test AC feasibility
* P in actual value, V in p.u.
* Use P to recover $\theta$, or solve $\theta$ and Q for PF
$$ Q_m = V_m \sum_{n=1}^N V_n \left(G_{mn}\sin\theta_{mn} - B_{mn}\cos\theta_{mn} \right) $$
calculate $Q_{mn}$ directly

In [ ]:
# Bbus and B_r inverse
filename1 = root+'data_1354ac_2022/ieee1354_Bbus.txt'
Bbus=pd.read_table(filename1,sep=',',header=None).to_numpy()
B_r = np.delete(Bbus,639,axis=0)
B_r = np.delete(B_r,639,axis=1)
Br_inv = np.linalg.inv(B_r)

# Jacobian for fast decoupled power flow 
filename1 = root+'data_1354ac_2022/ieee1354_ieee1354_J1.txt'
J1=pd.read_table(filename1,sep=',',header=None).to_numpy()
# J1_r = np.delete(J1,639,axis=0) # already reduced
# J1_r = np.delete(J1_r,639,axis=1)
J1r_inv = np.linalg.inv(J1)

# nominal angle for fast decoupled power flow 
filename1 = root+'data_1354ac_2022/ieee1354_ieee1354_theta_nominal.txt'
theta_nominal=pd.read_table(filename1,sep=',',header=None).to_numpy()
# nominal injection for fast decoupled power flow 
filename1 = root+'data_1354ac_2022/ieee1354_ieee1354_pinj_nominal.txt'
pinj_nominal=pd.read_table(filename1,sep=',',header=None).to_numpy()

# line parameters
filename1 = root+'data_1354ac_2022/ieee1354_lineloc.txt'
line_loc = pd.read_table(filename1,sep=',',header=None).to_numpy()
filename1 = root+'data_1354ac_2022/ieee1354_lineparams_tap.txt'
line_params = pd.read_table(filename1,sep=',',header=None).to_numpy()

n_line = 1751 # the rest are transformers
line_loc = line_loc[:n_line,:].copy()
line_params = line_params[:n_line,:].copy()

R_line = line_params[:,0].copy()
X_line = line_params[:,1].copy()

B_shunt = line_params[:,2].copy()

Z_line = R_line + 1j * X_line 
Y_line = 1 / Z_line

G_line = np.real(Y_line)
B_line = np.imag(Y_line)

# transformer indicator
# a = R_line > 0
a = np.ones(n_line)
a=torch.from_numpy(a)
G_line=torch.from_numpy(G_line)
B_line=torch.from_numpy(B_line)
B_shunt=torch.from_numpy(B_shunt)

# P_inj w/out reference bus in p.u.
p_inj_r = np.delete(p_inj,639,axis=0) / 100
p_inj_true_r = np.delete(p_inj_true,639,axis=0) / 100

p_inj_pu = p_inj / 100
p_inj_true_pu = p_inj_true / 100

# print(Br_inv.shape,p_inj.shape,p_inj_true.shape)#p_inj_true
print(J1r_inv.shape,p_inj.shape,p_inj_true.shape)
print(J1r_inv.shape,p_inj_r.shape,pinj_nominal.shape)

theta0 = np.matmul(J1r_inv,p_inj_r-pinj_nominal)
theta_true0 = np.matmul(J1r_inv,p_inj_true_r-pinj_nominal)

theta = np.insert(theta0,639,0,axis = 0)
theta_true = np.insert(theta_true0,639,0,axis = 0)
# restore from deviation
theta = theta + theta_nominal/180*math.pi
theta_true = theta_true + theta_nominal/180*math.pi
print(theta.shape,theta_true.shape)

In [ ]:
# B_line
n_sample1 = n_sample
f_tot_sample1 = n_line*n_sample1
# load true voltage file
filename1 = root+'data_1354ac_2022/ieee1354pegase_ac_9974_cases_v_04022022.txt'
v_true = pd.read_table(filename1,sep=',',header=None).to_numpy()
print(v_true.shape)
# load true theta file
filename2 = root+'data_1354ac_2022/ieee1354pegase_ac_9974_cases_theta_04022022.txt'
theta_true = pd.read_table(filename2,sep=',',header=None).to_numpy()
# theta_true = theta_true / 180 * math.pi
print(theta_true)
print(theta_true.shape)

v_true = np.delete(v_true, del_idx,axis=1)
theta_true = np.delete(theta_true, del_idx,axis=1)
print(v_true.shape,theta_true.shape)

In [ ]:
# print(theta)
# print('')
# print(theta_true)
v_pred = y_pred1[:,1,:].copy()
v_pred = v_pred / vy_scale + vy_deviation
print(v_pred)
print('')
print(v_true)

In [ ]:
# Calculate real and reactive flow
f_p = np.zeros((n_line,n_sample1))
f_q = np.zeros((n_line,n_sample1))
fji_p = np.zeros((n_line,n_sample1))
fji_q = np.zeros((n_line,n_sample1))
print(f_q.shape)

v_pred = y_pred1[:,1,:].copy()
v_pred = v_pred / vy_scale + vy_deviation

print(np.max(v_pred),np.min(v_pred),v_pred.shape)

theta1 = theta[line_loc[:,0]-1,:]
theta2 = theta[line_loc[:,1]-1,:]
V1 = torch.from_numpy(v_pred[line_loc[:,0]-1,:]).double()
V2 = torch.from_numpy(v_pred[line_loc[:,1]-1,:]).double() 
theta1=torch.from_numpy(theta1)
theta2=torch.from_numpy(theta2)

f_p=(a*G_line*(V1*V1).T)-a*((V1*V2).T)*(G_line*torch.cos(theta1-theta2).T+B_line*torch.sin(theta1-theta2).T)
f_p=f_p.T
f_q=-a*(V1.T)*(a*V1.T)*(B_line+B_shunt/2)+a*((V1*V2).T)*(B_line*torch.cos(theta1-theta2).T-G_line*torch.sin(theta1-theta2).T)
f_q=f_q.T

theta1 = theta[line_loc[:,1]-1,:]
theta2 = theta[line_loc[:,0]-1,:]
V1 = torch.from_numpy(v_pred[line_loc[:,1]-1,:]).double()
V2 = torch.from_numpy(v_pred[line_loc[:,0]-1,:]).double()
theta1=torch.from_numpy(theta1)
theta2=torch.from_numpy(theta2)

fji_p=(a*G_line*(V1*V1).T)-a*((V1*V2).T)*(G_line*torch.cos(theta1-theta2).T+B_line*torch.sin(theta1-theta2).T)
fji_p=fji_p.T
fji_q=-a*(V1.T)*(a*V1.T)*(B_line+B_shunt/2)+a*((V1*V2).T)*(B_line*torch.cos(theta1-theta2).T-G_line*torch.sin(theta1-theta2).T)
fji_q=fji_q.T

In [ ]:
print('mean p_ij,q_ij:',torch.mean(f_p),torch.mean(f_q))
print('mean p_ji,q_ji:',torch.mean(fji_p),torch.mean(fji_q))

In [ ]:
theta = theta_true.copy()
print('Scale check:')
i=0;j=0
a = np.ones(1751)
print('G,B,loc:',G_line[i],B_line[i],line_loc[i,0]-1,line_loc[i,1]-1) # G,B,line_loc verified
print('v:',v_pred[line_loc[i,0]-1,j],v_pred[line_loc[i,1]-1,j]) # v verified
print('theta:',theta[line_loc[i,0]-1,j],theta[line_loc[i,1]-1,j]) # theta verified
print('p,q:',f_p[0,0],f_q[0,0],fji_p[i,j],fji_q[i,j])

f_p_test = a[i] * v_pred[line_loc[i,0]-1,j] * v_pred[line_loc[i,0]-1,j] * G_line[i] \
             - (a[i] * v_pred[line_loc[i,0]-1,j]) * v_pred[line_loc[i,1]-1,j] \
             * (G_line[i] * math.cos(theta[line_loc[i,0]-1,j] - theta[line_loc[i,1]-1,j]) \
              + B_line[i] * math.sin(theta[line_loc[i,0]-1,j] - theta[line_loc[i,1]-1,j]))
             
f_q_test =-(a[i] * v_pred[line_loc[i,0]-1,j]) **2 * (B_line[i] + B_shunt[i]/2) \
             + a[i] * v_pred[line_loc[i,0]-1,j] * v_pred[line_loc[i,1]-1,j] \
             * (B_line[i] * math.cos(theta[line_loc[i,0]-1,j] - theta[line_loc[i,1]-1,j]) \
              - G_line[i] * math.sin(theta[line_loc[i,0]-1,j] - theta[line_loc[i,1]-1,j]))
             
print('test p/q:',f_p_test,f_q_test)
print(a[i],v_pred[line_loc[i,0]-1,j],B_line[i] + B_shunt[i]/2,v_pred[line_loc[i,1]-1,j])
print(G_line[i],math.cos(theta[line_loc[i,0]-1,j] - theta[line_loc[i,1]-1,j]),math.sin(theta[line_loc[i,0]-1,j] - theta[line_loc[i,1]-1,j]))

In [ ]:
s_pred = torch.sqrt(f_p*f_p+f_q*f_q)*100
sji_pred = torch.sqrt(fji_p*fji_p+fji_q*fji_q)*100
f_p = f_p.numpy();f_q = f_q.numpy()
s_pred = s_pred.numpy();sji_pred = sji_pred.numpy()
print(np.max(f_q),np.min(f_q))
flow_est.shape
print(np.mean(s_pred[0,:]),np.mean(f_max_numpy[0]))

In [ ]:
sij_binary = (np.abs(s_pred)-f_max_numpy[:n_line] > 0)
sji_binary = (np.abs(sji_pred)-f_max_numpy[:n_line] > 0)
s_binary = np.maximum(sij_binary,sji_binary)
print(np.sum(s_binary))#,np.sum(f_binary0))
print('hard violation rate:',np.sum(s_binary)/f_tot_sample1)#,np.sum(f_binary0)/f_tot_sample)
s_binary_soft = (np.abs(s_pred)-f_max_numpy[:n_line] > 0.1*(f_max_numpy[:n_line]))
print(np.sum(s_binary_soft))#,np.sum(f_binary0_soft))
print(np.sum(s_binary_soft)/f_tot_sample1)#,np.sum(f_binary0_soft)/f_tot_sample)

In [ ]:
# violation level
sij_violation = np.abs(s_pred)-f_max_numpy[:n_line] #/ f_max_numpy
sij_violation_level = np.maximum(sij_violation,0)
sji_violation = np.abs(sji_pred)-f_max_numpy[:n_line] #/ f_max_numpy
sji_violation_level = np.maximum(sji_violation,0)
s_violation_level = np.maximum(sij_violation_level,sji_violation_level)
s_violation_level = np.divide(s_violation_level,f_max_numpy[:n_line])
s_vio_lvl = np.reshape(s_violation_level,n_line*n_sample1)

print('S violation level:')
print('hard:',np.sum(s_binary)/f_tot_sample1)
print('mean:',np.mean(s_vio_lvl))
print('median:',np.median(s_vio_lvl))
print('max:',np.max(s_vio_lvl))
print('std:',np.std(s_vio_lvl))
print('p99:',np.percentile(s_vio_lvl,99))

f_violation = np.abs(flow_est)-f_max_numpy #/ f_max_numpy
f_violation_level = np.maximum(f_violation,0)
f_violation_level = np.divide(f_violation_level,f_max_numpy)
f_vio_lvl = np.reshape(f_violation_level,1991*n_sample)

print('f violation level:')
print('hard:',np.sum(f_binary)/f_tot_sample,np.sum(f_binary0)/f_tot_sample)
print('mean:',np.mean(f_vio_lvl))
print('median:',np.median(f_vio_lvl))
print('max:',np.max(f_vio_lvl))
print('std:',np.std(f_vio_lvl))
print('p99:',np.percentile(f_vio_lvl,99))

# fig4 = plt.figure(figsize=(6,4))
# plt.hist(s_vio_lvl, bins = 50, facecolor='b', alpha=0.75,label = 's violation')
# plt.hist(f_vio_lvl, bins = 50, facecolor='r', alpha=0.75,label = 'f violation')
# plt.legend(loc="upper right")
# plt.xlabel('violation level')
# plt.ylabel('frequency')
# # plt.title('injection histogram')
# plt.show()

In [ ]:
# err_L2_mean = np.mean(err_L2)
# err_Linf_mean = np.mean(err_Linf)
print('Price L2 mean:', err_L2_mean,'L_inf mean:', err_Linf_mean )
print('std:',np.std(err_L2))
# err_L2_mean_v = np.mean(err_L2_v)
# err_Linf_mean_v = np.mean(err_Linf_v)
print('Voltage L2 mean:', err_L2_mean_v,'L_inf mean:', err_Linf_mean_v )
print('std:',np.std(err_L2_v))